In [2]:
#tipicas
import pandas as pd
import numpy as np

#para la creacion de carpetas
import os
import shutil

#para el test-train split
import splitfolders #(necesita pip install split-folders)

In [ ]:
#importamos el df con las labels
path_csv = "/home/vvdiaz1/code/vvdiaz1/automatic_tagging/raw_data/filterted_small_final.csv"
df = pd.read_csv(path_csv,delimiter=';')

In [ ]:
df[df.subCategory == 'Scarves and Stoles']

In [ ]:
# img_names = os.listdir("/home/vvdiaz1/code/vvdiaz1/automatic_tagging/raw_data/data_small")
# list_id = []
# for i in img_names:
#     list_id.append(int(i[:-4]))
# len(list_id)

In [ ]:
# aux = df.drop(df.columns.difference(['id', 'masterCategory','subCategory','articleType']),1)
# aux = aux[aux.id.isin(list_id)]
# aux['name'] = aux['id'].apply(str) +'.jpg'
# aux

In [ ]:
#guarda el csv
# aux.to_csv('filtered_small.csv',index = False)

In [10]:
input_folder = "/home/vvdiaz1/code/vvdiaz1/automatic_tagging/raw_data/data_sort_1/"
output_folder = "/home/vvdiaz1/code/vvdiaz1/automatic_tagging/raw_data/data_small_3"

In [11]:
splitfolders.ratio(input_folder, output=output_folder,
                    seed=42,
                    ratio=(.14, .8, .06), #usamos 70% train, 20% val, 10% test 
                    group_prefix=None)

In [2]:
import keras
import numpy as np
import pickle
import os
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, Input
from keras.initializers import he_normal
from keras import optimizers
from keras.callbacks import LearningRateScheduler, TensorBoard
from keras.layers import BatchNormalization
from keras.utils.data_utils import get_file
from keras import backend as K

2023-01-28 16:20:58.392424: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-28 16:21:03.375459: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-28 16:21:03.375634: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-01-28 16:21:09.703016: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [ ]:
data_dir = "/home/vvdiaz1/code/vvdiaz1/automatic_tagging/raw_data/data_small"

In [ ]:
def scheduler(epoch):
  learning_rate_init = 0.001
  if epoch > 55:
    learning_rate_init = 0.0002
  if epoch > 70:
    learning_rate_init = 0.00005
  return learning_rate_init

def unpickle(filename):
  file = os.path.join(data_dir, filename)
  with open(file, 'rb') as fo:
    dict = pickle.load(fo, encoding='bytes')
  return dict

class LossWeightsModifier(keras.callbacks.Callback):
  def __init__(self, alpha, beta, gamma):
    self.alpha = alpha
    self.beta = beta
    self.gamma = gamma
    # customize your behavior
  def on_epoch_end(self, epoch, logs={}):
    if epoch == 13:
      K.set_value(self.alpha, 0.1)
      K.set_value(self.beta, 0.8)
      K.set_value(self.gamma, 0.1)
    if epoch == 23:
      K.set_value(self.alpha, 0.1)
      K.set_value(self.beta, 0.2)
      K.set_value(self.gamma, 0.7)
    if epoch == 33:
      K.set_value(self.alpha, 0)
      K.set_value(self.beta, 0)
      K.set_value(self.gamma, 1)

In [ ]:
#-------- dimensions ---------
height, width = 480, 360
channel = 3
if K.image_data_format() == 'channels_first':
    input_shape = (channel, height, width)
else:
    input_shape = (height, width, channel)

In [ ]:
train_size = 7084
test_size = 1771

#--- coarse 1 classes ---
coarse1_classes = 4 #full es el mismo
#--- coarse 2 classes ---
coarse2_classes = 37 #full es 38
#--- fine classes ---
num_classes  = 123 #full es 136

batch_size   = 32
epochs       = 20

In [ ]:
#carga de vgg16
def load_model():
    """Cargamos el modelo pre-entrenado"""

    model = VGG16(weights="imagenet", include_top=False, input_shape = (480,360,3))
    
    return model

def set_nontrainable_layers(model):
    """Establecemos las primeras capas como no entrenables"""

    model.trainable = False
    
    return model

def add_last_layers(model):
    """Tomamos un modelo pre-entrenado, establecemos los parametros como no entrenables, 
    y añadimos capas entrenables adicionales en la parte superior"""

    base_model = set_nontrainable_layers(model)
    flatten_layer = layers.Flatten()
    dense_layer = layers.Dense(500, activation='relu')
    prediction_layer = layers.Dense(num_cat2, activation='softmax') #CUIDADO AQUI! USAMOS num_cat2 para que todo coincida
    
    model = models.Sequential([
        base_model,
        flatten_layer,
        dense_layer,
        prediction_layer
    ])
    
    return model

def build_model():
    """Traemos todas la funciones en una"""
    
    model = load_model()
    model = add_last_layers(model)
    
    opt = optimizers.Adam(learning_rate=1e-4)
    model.compile(loss='categorical_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])
    return model

In [ ]:
model = VGG16(weights="imagenet", include_top=False, input_shape = (480,360,3))

In [1]:
from keras.preprocessing.image import ImageDataGenerator
import numpy as np

# Create an ImageDataGenerator object
data_gen = ImageDataGenerator(rescale = 1./255)

# Generate batches of image data
gen = data_gen.flow_from_directory("/home/vvdiaz1/code/vvdiaz1/automatic_tagging/raw_data/data_small_3/train",
                             target_size = (480,360),batch_size=15)

# Initialize an empty list to store the batches
image_list = []

# Generate batches of data and append them to the list
for x, y in gen:
    image_list.append(x)

# Concatenate the batches to create a single numpy array
image_array = np.concatenate(image_list)

2023-01-28 15:38:57.441218: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-28 15:39:05.259546: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-28 15:39:05.259746: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-01-28 15:39:15.337701: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

Found 6198 images belonging to 4 classes.


: 

: 

In [ ]:
x_train = image_array

In [4]:
import pandas as pd
path_csv = "/home/vvdiaz1/code/vvdiaz1/automatic_tagging/raw_data/filtered_full_final.csv"
df = pd.read_csv(path_csv,delimiter=';')

In [7]:
import os
img_names = os.listdir("/home/vvdiaz1/code/vvdiaz1/automatic_tagging/raw_data/data_small_2/test")
list_id = []
for i in img_names:
    list_id.append(int(i[:-4]))
len(list_id)

2661

In [8]:
aux = df.drop(df.columns.difference(['id', 'masterCategory','subCategory','articleType']),1)
aux = aux[aux.id.isin(list_id)]
aux['name'] = aux['id'].apply(str) +'.jpg'
aux

/tmp/ipykernel_7486/2301221239.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  aux = df.drop(df.columns.difference(['id', 'masterCategory','subCategory','articleType']),1)


,id,masterCategory,subCategory,articleType,name
1,39386,Apparel,Bottomwear,Jeans,39386.jpg
11,48123,Accessories,Belts,Belts,48123.jpg
76,54118,Footwear,Shoes,Heels,54118.jpg
86,44970,Accessories,Watches,Watches,44970.jpg
132,7964,Apparel,Topwear,Tshirts,7964.jpg
...,...,...,...,...,...
44239,56250,Apparel,Loungewear and Nightwear,Night suits,56250.jpg
44250,23031,Apparel,Topwear,Shirts,23031.jpg
44251,17204,Accessories,Eyewear,Sunglasses,17204.jpg
44254,45223,Accessories,Watches,Watches,45223.jpg


In [9]:
aux.to_csv('filtered_test.csv',index = False)

In [18]:
from tensorflow.keras.utils import to_categorical

#para el modelo 1 estas son las labels = masterCategory
labels1 = np.unique(aux.masterCategory).tolist()
num_cat1 = len(labels1)
# print('Categories: ',labels1)
# print('Number of categories: ', num_cat1)

#definimos y como el masterCategory y luego los convertimos a int para usar to_categorical
y1 = aux[['masterCategory']]
y1.masterCategory =pd.Categorical(y1.masterCategory)
y1_num = y1.masterCategory.cat.codes
y1_cat = to_categorical(y1_num,num_cat1)

#revisamos
# print(len(y1_cat[0]) == num_cat1)

#para el modelo 1 estas son las labels = masterCategory
labels1 = np.unique(aux.subCategory).tolist()
num_cat1 = len(labels1)
# print('Categories: ',labels1)
# print('Number of categories: ', num_cat1)

#definimos y como el masterCategory y luego los convertimos a int para usar to_categorical
y1 = aux[['subCategory']]
y1.subCategory =pd.Categorical(y1.subCategory)
y1_num = y1.subCategory.cat.codes
y2_cat = to_categorical(y1_num,num_cat1)

#revisamos
# print(len(y2_cat[0]) == num_cat1)

#para el modelo 1 estas son las labels = masterCategory
labels1 = np.unique(aux.articleType).tolist()
num_cat1 = len(labels1)
# print('Categories: ',labels1)
# print('Number of categories: ', num_cat1)

#definimos y como el masterCategory y luego los convertimos a int para usar to_categorical
y1 = aux[['articleType']]
y1.articleType =pd.Categorical(y1.articleType)
y1_num = y1.articleType.cat.codes
y3_cat = to_categorical(y1_num,num_cat1)

#revisamos
# print(len(y3_cat[0]) == num_cat1)

/tmp/ipykernel_17128/3943656411.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y1.masterCategory =pd.Categorical(y1.masterCategory)
/tmp/ipykernel_17128/3943656411.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y1.subCategory =pd.Categorical(y1.subCategory)
/tmp/ipykernel_17128/3943656411.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.p

In [19]:
y3_cat.shape

(6198, 120)

In [ ]:
#-------------------- data loading ----------------------
x_train = np.reshape(train[b'data'], (train_size, channel, height, width)).transpose(0, 2, 3, 1).astype("float32")
x_train = (x_train-np.mean(x_train)) / np.std(x_train)

x_test = np.reshape(test[b'data'], (test_size, channel, height, width)).transpose(0, 2, 3, 1).astype("float32")
x_test = (x_test-np.mean(x_test)) / np.std(x_test)

y_train = np.zeros((train_size, num_classes)).astype('float32')
y_c2_train = np.zeros((train_size, coarse2_classes)).astype('float32')

y_test = np.zeros((test_size, num_classes)).astype('float32')
y_c2_test = np.zeros((test_size, coarse2_classes)).astype('float32')

y_train[np.arange(train_size), train[b'fine_labels']] = 1
y_c2_train[np.arange(train_size), train[b'coarse_labels']] = 1

y_test[np.arange(test_size), test[b'fine_labels']] = 1
y_c2_test[np.arange(test_size), test[b'coarse_labels']] = 1

c2_to_f = np.zeros((coarse2_classes, num_classes)).astype('float32')
fine_unique, fine_unique_indices = np.unique(train[b'fine_labels'], return_index=True)
for i in fine_unique_indices:
  c2_to_f[train[b'coarse_labels'][i]][train[b'fine_labels'][i]] = 1

parent_c2 = {
  0:0, 1:0, 2:1, 3:2, 
  4:1, 5:2, 6:2, 7:3, 
  8:4, 9:5, 10:5, 11:4, 
  12:4, 13:3, 14:6, 15:4, 
  16:4, 17:1, 18:7, 19:7
}

y_c1_train = np.zeros((y_c2_train.shape[0], coarse1_classes)).astype("float32")
y_c1_test = np.zeros((y_c2_test.shape[0], coarse1_classes)).astype("float32")
for i in range(y_c1_train.shape[0]):
  y_c1_train[i][parent_c2[np.argmax(y_c2_train[i])]] = 1.0
for i in range(y_c1_test.shape[0]):
  y_c1_test[i][parent_c2[np.argmax(y_c2_test[i])]] = 1.0

del(train)
del(test)